###Modélisation énergétique sous OMEGAlpes

Tout d'abord, nous allons importer tous les élements nécessaires à la construction de notre modèle.

In [ ]:
from examples.utils.data_utils import *

Un premier point pour étudier la stratégie de gestion énergétique optimale de cette
maison est d’introduire la dynamique associée au cas d’étude. Ici, nous choisirons d’étudier un scénario d’une période de 24 heures, au pas de temps de 5 minutes. De plus, le cas d’étude est appliqué à la journée du 10 août 2016. Ces informations nous permettent d’instancier notre classe *TimeUnit* comme suit :

In [2]:
time = TimeUnit(start='10/08/2016', periods=24*12, dt=1/12)

You are studying the period from 2016-08-10 00:00:00 to 2016-08-10 23:55:00


Ici, le nombre de périodes, défini par le paramètre *periods*, est de 24 heures au pas de temps 5 minutes, donc 24\*12 périodes. Le pas de temps *dt* est défini de façon horaire et fixé à 1 par défaut. Ainsi, notre pas de temps de 5 minutes est défini comme 1/12 d'heure. Il aurait également été possible de définir notre objet représentatif du temps en définissant des dates de début et de fin à la place du nombre de périodes. Par défaut, la date de début du problème est le 01/01/2018.

Ensuite, nous pouvons passer à la description énergétique du cas d'étude. Côté réseau électrique, l'import et l'export d'énergie ont été respectivement représentés par des unités de production et de consommation variables, puisque les flux énergétiques associés dépendront de la stratégie de consommation établie. Les lignes de code associées sont donc les suivantes :

In [3]:
exports = VariableConsumptionUnit(time, name="exports", 
                                  energy_type=energy_types.elec)

Creating the exports.
Creating the exports.


In [4]:
imports = VariableProductionUnit(time, name="imports", pmax=6000,
                                 energy_type=energy_types.elec)

Creating the imports.
Creating the imports.


À l'import énergétique du réseau électrique, s'ajoute la production solaire photovoltaïque (PV). Non pilotable, elle sera donc modélisée grâce à une unité de production fixe, en entrant son profil de production (PV_profile), comme suit :

In [ ]:
PV_profile = import_PV_profile_5_min()

Importing PV profile at a 5-minutes time step.


FileNotFoundError: File b'data/PV_production.csv' does not exist

In [6]:
PV_prod = FixedProductionUnit(time, name="PV_prod", p=PV_profile, 
                              energy_type=energy_types.elec)

Creating the PV_prod.
Creating the PV_prod.


De façon similaire, la consommation d'eau chaude sanitaire est considérée comme connue sur l'ensemble de la journée. Cette unité peut donc être définie par une unité de consommation fixe, grâce au profil de consommation (dhw_load), avec la commande suivante :

In [7]:
dhw_load = import_domestic_hot_water_load_profile()
dhw = FixedConsumptionUnit(time, name="dhw", p=dhw_load, 
                                energy_type=energy_types.heat)

Creating the dhw.
Creating the dhw.


Les deux autres charges (électriques) sont considérées comme flexibles du point de vue du moment de démarrage. Les profils sont donc connus, mais peuvent cette fois être déplacés dans le temps. On utilisera les unités de consommation *ShiftableConsumptionUnit*, paramétrées par les profils respectifs de la machine à laver (washer_load) et du sèche-linge (dryer_load), comme suit :

In [8]:
washer_load, dryer_load = import_clothes_washer_and_dryer_load_profiles()
washer = ShiftableConsumptionUnit(time, name="washer", 
                                  power_values=washer_load,  
                                  energy_type=energy_types.elec)

dryer = ShiftableConsumptionUnit(time, name="dryer", 
                                 power_values=dryer_load,  
                                 energy_type=energy_types.elec)

Creating the washer.
Creating the washer.
Creating the washer.
Creating the dryer.
Creating the dryer.
Creating the dryer.


Dernière consommation électrique, le chauffe-eau électrique convertit de 90% de l'électricité consommée en énergie thermique. Il sera donc représenté par une unité de conversion énergétique, et plus particulièrement, une unité de conversion d'énergie électrique en énergie thermique avec un facteur de conversion constant, selon la ligne de code suivante :

In [9]:
water_heater = ElectricalToHeatConversionUnit(
    time, name="water_heater", elec_to_heat_ratio=0.9)

Creating the water_heater_heat_prod.
Creating the water_heater_heat_prod.
Creating the water_heater_elec_cons.
Creating the water_heater_elec_cons.
Creating the water_heater.


Enfin, il nous reste à intégrer le ballon d'eau chaude. Pour cela, nous nous appuierons sur un modèle simple de stockage d'une capacité de 6 kWh, ne devant pas se vider en-dessous de 20% de sa capacité et étant imposé à revenir à son état de charge initial à la fin de la journée. Nous ajouterons également un taux d'auto-décharge de 5% par heure. La formulation associée est la suivante :

In [10]:
water_tank = StorageUnit(time, name="water_tank",  self_disch=0.05, 
                         soc_min=0.2, energy_type=energy_types.heat,
                         ef_is_e0=True, capacity=6000, )

Creating the water_tank.


Une fois les unités énergétiques définies, elles peuvent être connectées entre elles par l'intermédiaire des noeuds énergétiques. Dans notre cas, nous avons un bilan  électrique et un bilan thermique, donc deux noeuds énergétiques, définis comme :

In [11]:
elec_node = EnergyNode(time, name="elec_node", 
                       energy_type=energy_types.elec)

Creating the elec_node.


In [12]:
heat_node = EnergyNode(time, name="heat_node", 
                       energy_type=energy_types.heat)

Creating the heat_node.


La connexion des unités énergétiques aux noeuds associés se fait alors comme suit :

In [13]:
elec_node.connect_units(imports, exports, PV_prod, washer, 
                        dryer, water_heater.elec_consumption_unit)

In [14]:
heat_node.connect_units(dhw, water_tank, 
                        water_heater.heat_production_unit)

### Ajout des contraintes externes et de l'objectif de modélisation


Dans ce cas d'étude, une contrainte externe doit être imposée : le fait que le sèche linge ne puisse pas démarrer tant que la machine à laver n'a pas fini son cycle. La contrainte étant ici temporelle, on définira cette dernière grâce à la classe *ExtDynConstraint*, héritant des classes de contraintes externe et dynamique, comme suit :

In [15]:
cst = ExtDynConstraint(name="wait_to_dry", 
                       exp_t="dryer_start_up[t] <= "
                              "lpSum(washer_switch_off[k] "
                              "for k in range(0, t))",
                       parent=dryer)


La contrainte étant définie à partir de la variable d'arrêt de la machine à laver, il est nécessaire d'imposer que cette varible soit calculée, grâce à la commande suivante :

In [16]:
washer._add_switch_off()

Une fois définie, la contrainte est associée au sèche-linge de la façon suivante :


In [17]:
setattr(dryer, "wait_to_dry", cst)

Il est à noter que nous considérerons ici qu'une personne est présente dans l'habitation, afin de réaliser le transfert de vêtements. De problèmes plus complexes pourraient être modélisés en intégrant des contraintes externes limitant la plage horaire durant laquelle il est possible de transférer le linge.


Désormais, nous souhaitons ajouter notre objectif : maximiser l'autoconsommation sur la journée, i.e. la part de la production d'électricité produite par les panneaux PV qui est consommée sur par la maison individuelle. Pour cela, nous cherchons donc à minimiser l'apport du réseau électrique et utiliserons donc la commande *minimize_production*, de la façon suivante :

In [18]:
imports.minimize_production()

### Lancement de l'optimisation et résultats

Maintenant que le modèle énergétique est prêt et que les contrainte et objectif ont été ajoutés, nous souhaitons réaliser l'optimisation.


La première étape est donc de créer le méta-modèle d'optimisation sous OMEGAlpes et d'y ajouter les noeuds énergétiques :


In [19]:
model = OptimisationModel(time, name="example")

In [20]:
model.add_nodes(elec_node, heat_node)


--- Adding all variables to the model ---
Adding variable : imports_p
Adding variable : imports_e_tot
Adding variable : imports_u
Adding variable : exports_p
Adding variable : exports_e_tot
Adding variable : exports_u
Adding variable : PV_prod_p
Adding variable : PV_prod_e_tot
Adding variable : washer_p
Adding variable : washer_e_tot
Adding variable : washer_u
Adding variable : washer_start_up
Adding variable : washer_switch_off
Adding variable : washer_power_values
Adding variable : dryer_p
Adding variable : dryer_e_tot
Adding variable : dryer_u
Adding variable : dryer_start_up
Adding variable : dryer_power_values
Adding variable : water_heater_elec_cons_p
Adding variable : water_heater_elec_cons_e_tot
Adding variable : water_heater_elec_cons_u
Adding variable : water_heater_heat_prod_p
Adding variable : water_heater_heat_prod_e_tot
Adding variable : water_heater_heat_prod_u
Adding variable : dhw_p
Adding variable : dhw_e_tot
Adding variable : water_tank_p
Adding variable : water_tan


Adding constraint : washer_def_0_power_value , exp = washer_p[t] >= washer_power_values[0] * washer_start_up[t-0] for t in time.I[0:-1]
Adding constraint : washer_def_1_power_value , exp = washer_p[t] >= washer_power_values[1] * washer_start_up[t-1] for t in time.I[1:-1]
Adding constraint : washer_def_2_power_value , exp = washer_p[t] >= washer_power_values[2] * washer_start_up[t-2] for t in time.I[2:-1]
Adding constraint : washer_def_3_power_value , exp = washer_p[t] >= washer_power_values[3] * washer_start_up[t-3] for t in time.I[3:-1]
Adding constraint : washer_def_4_power_value , exp = washer_p[t] >= washer_power_values[4] * washer_start_up[t-4] for t in time.I[4:-1]
Adding constraint : washer_def_5_power_value , exp = washer_p[t] >= washer_power_values[5] * washer_start_up[t-5] for t in time.I[5:-1]
Adding constraint : washer_def_6_power_value , exp = washer_p[t] >= washer_power_values[6] * washer_start_up[t-6] for t in time.I[6:-1]
Adding constraint : washer_def_switch_off , exp

Adding constraint : dryer_def_9_power_value , exp = dryer_p[t] >= dryer_power_values[9] * dryer_start_up[t-9] for t in time.I[9:-1]
Adding constraint : dryer_wait_to_dry , exp = dryer_start_up[t] <= lpSum(washer_switch_off[k] for k in range(0, t)) for t in time.I
Adding constraint : water_heater_elec_cons_calc_e_tot , exp = water_heater_elec_cons_e_tot == time.DT * lpSum(water_heater_elec_cons_p[t] for t in time.I)
Adding constraint : water_heater_elec_cons_on_off_max , exp = water_heater_elec_cons_p[t] <= water_heater_elec_cons_u[t] * 100000.0 for t in time.I


Adding constraint : water_heater_elec_cons_on_off_min , exp = water_heater_elec_cons_p[t] >= water_heater_elec_cons_u[t] * 1e-05 for t in time.I
Adding constraint : water_heater_conversion , exp = water_heater_heat_prod_p[t] == 0.9 * water_heater_elec_cons_p[t] for t in time.I
Adding constraint : water_heater_heat_prod_calc_e_tot , exp = water_heater_heat_prod_e_tot == time.DT * lpSum(water_heater_heat_prod_p[t] for t in time.I)
Adding constraint : water_heater_heat_prod_on_off_max , exp = water_heater_heat_prod_p[t] <= water_heater_heat_prod_u[t] * 100000.0 for t in time.I
Adding constraint : water_heater_heat_prod_on_off_min , exp = water_heater_heat_prod_p[t] >= water_heater_heat_prod_u[t] * 1e-05 for t in time.I
Adding constraint : heat_node_power_balance , exp = -dhw_p[t]-water_tank_p[t]+water_heater_heat_prod_p[t] == 0 for t in time.I
Adding constraint : dhw_calc_e_tot , exp = dhw_e_tot == time.DT * lpSum(dhw_p[t] for t in time.I)
Adding constraint : water_tank_calc_e_tot , exp 

Adding constraint : water_tank_def_max_charging , exp = water_tank_pc[t] - water_tank_uc[t] * 100000.0 <= 0 for t in time.I
Adding constraint : water_tank_def_max_discharging , exp = water_tank_pd[t] - (1 - water_tank_uc[t]) * 100000.0 <= 0 for t in time.I
Adding constraint : water_tank_def_min_charging , exp = water_tank_pc[t] - water_tank_uc[t] * 1e-05 >= 0 for t in time.I
Adding constraint : water_tank_def_min_discharging , exp = water_tank_pd[t] + (water_tank_uc[t] - water_tank_u[t]) * 1e-05 >= 0 for t in time.I
Adding constraint : water_tank_e_f_min , exp = water_tank_e_f >= 0.2 * water_tank_capacity
Adding constraint : water_tank_e_f_max , exp = water_tank_e_f <= 1 * water_tank_capacity
Adding constraint : water_tank_calc_e_f , exp = water_tank_e_f-water_tank_e[287] == 0.08333333333333333*(water_tank_pc[287]*1-water_tank_pd[287]*1/1-0*water_tank_e[287]-0.05*water_tank_capacity)
Adding constraint : water_tank_ef_is_e0 , exp = water_tank_e[0] == water_tank_e_f


In [21]:
from omegalpes.general.plots import plot_node_energetic_flows

In [22]:
model.solve_and_update()


 - - - - - RUN OPTIMIZATION - - - - - 


Resolution duration = 208.0644016265869 seconds.

 - - - - - UPDATE RESULTS - - - - - 
Updating unit : elec_node
Updating unit : imports
	Quantity : p
	Quantity : e_tot
	Quantity : u
Updating unit : exports
	Quantity : p
	Quantity : e_tot
	Quantity : u
Updating unit : PV_prod
	Quantity : p
	Quantity : e_tot
Updating unit : washer
	Quantity : p
	Quantity : e_tot
	Quantity : u
	Quantity : start_up
	Quantity : switch_off
	Quantity : power_values
Updating unit : dryer
	Quantity : p
	Quantity : e_tot
	Quantity : u
	Quantity : start_up
	Quantity : power_values
Updating unit : water_heater_elec_cons
	Quantity : p
	Quantity : e_tot
	Quantity : u
Updating unit : water_heater
Updating unit : water_heater_heat_prod
	Quantity : p
	Quantity : e_tot
	Quantity : u
Updating unit : heat_node
Updating unit : dhw
	Quantity : p
	Quantity : e_tot
Updating unit : water_tank
	Quantity : p
	Quantity : e_tot
	Quantity : u
	Quantity : capacity
	Quantity : e
	Quantity : pc
	Quantity : pd
	Quantity : uc
	Quantity

Il est désormais possible de visualiser les résultats. Certaines commandes peuvent être directement utilisées en les importants du module *plots* : 


In [23]:
from omegalpes.general.plots import *

In [ ]:
plot_node_energetic_flows(elec_node)

In [27]:
from omegalpes.general.utils import save_energy_flows

In [ ]:
save_energy_flows(elec_node, heat_node, sep=';',
                  file_name=os.getcwd() + 
                            'example_results.csv')